<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Profesor: Carlos Pérez </em><br>

</div>

# Trabajo Final Almacenamiento y Captura de Datos

Integrantes:
- Diego León
- Pablo Madariaga

# Introducción

El siguiente trabajo muestra el desarrollo de habilidades en Web Scraping y el uso de APIs para crear un producto de consulta de inmuebles y sus comercios cercanos. Se divide en tres partes principales:

- Interacción con páginas web: Utilizando Selenium, se deberá interactuar con un sitio de búsqueda de inmuebles, extraer información clave, convertir valores monetarios de UF a pesos chilenos y resolver problemas relacionados con direcciones incompletas.

- Uso de Google APIs: Con Geocoding API, se obtendrán coordenadas y el identificador de lugar de cada inmueble, mientras que con Places API se identificarán comercios cercanos.

- Gestión de datos: Se requiere crear una base de datos con SQLite para almacenar y relacionar los datos obtenidos, además de realizar consultas SQL específicas para analizar los datos.


## Librerias a utilizar

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
import time  
import time  
import requests
import re
import sqlite3

## Parte 1: Interacción con páginas web

### Inputs

In [2]:
API_KEY = " " #cambiar segun quien lo esté usando
URL = 'https://www.portalinmobiliario.com/'
URL_UF = 'https://www.valoruf.cl/'
TIPO_CONTRATO = 'arriendo' # venta, arriendo o arriendo_temporal
TIPO_INMUEBLE = 'departamentos' # dpto, casa u oficina.
UBICACION_INMUEBLE = 'las condes' # la comuna de la búsqueda
MONTO_MINIMO = 500000 # monto mínimo de la búsqueda
MONTO_MAXIMO = 1300000 # monto máximo de la búsqueda
CANT_PAGINAS = 2 # número de páginas a recorrer
RADIO_BUSQUEDA = '300' # radio (en metros) de búsqueda de lugares cercanos
BUSQUEDA_RUBROS = ['restaurant', 'grocery_or_supermarket'] # rubro de lugares cercanos

### 1.1  Búsqueda inicial

La única búsqueda en chat GPT fue para identificar dinámicamente la n-esima opción asociada
a la búsqueda de la comuna

In [3]:
tipo_contrato_dict = {
    "venta": ":R2l5r:-menu-list-option-242075",
    "arriendo": ":R2l5r:-menu-list-option-242073",
    "arriendo temporal": ":R2l5r:-menu-list-option-242074"
}

tipo_inmueble_dict = {
    "departamentos": ":R4l5r:-menu-list-option-MLC1472_242062",
    "casas": ":R4l5r:-menu-list-option-MLC1466_242060",
    "oficinas": ":R4l5r:-menu-list-option-MLC1478_242067",
    "parcelas": ":R4l5r:-menu-list-option-MLC1496_242070",
    "locales": ":R4l5r:-menu-list-option-MLC50610_242065",
    "terrenos": ":R4l5r:-menu-list-option-MLC152992_245004",
    "sitios": ":R4l5r:-menu-list-option-MLC50613_245008",
    "bodegas": ":R4l5r:-menu-list-option-MLC50564_245003",
    "industriales": ":R4l5r:-menu-list-option-MLC50617_245009",
    "agricolas": ":R4l5r:-menu-list-option-MLC50623_242059",
    "otros inmuebles": ":R4l5r:-menu-list-option-MLC1892_242068",
    "estacionamientos": ":R4l5r:-menu-list-option-MLC50620_242064",
    "loteos": ":R4l5r:-menu-list-option-MLC1493_245010",
}

### En la página inicial, realizamos lo siguiente:
 - Filtrar según botón de po contrato.
 - Filtrar según botón de po inmueble.
 - Filtrar según botón de comuna de búsqueda.
 - Elegir la primera sugerencia de búsqueda.
 - Hacer clic en botón BUSCAR.

In [5]:
# Configuramos el driver para el navegador Edge
#driver = webdriver.Edge()
driver = webdriver.Safari()
driver.set_window_size(1200, 800)  # Ancho: 1200px, Alto: 800px

driver.get(URL)

# Establecemos una espera implícita de 5 segundos para que la página cargue completamente
driver.implicitly_wait(10)

buscador_comuna = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, ":Rml5r:"))
)

# Ingresa una comuna o ciudad
buscador_comuna.send_keys(UBICACION_INMUEBLE)

ul_comunas = "andes-list.faceted-search-desktop-searchbox__list.andes-list--default.andes-list--selectable"
resultados = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CLASS_NAME, ul_comunas))
)
# seleccionar primera opcion de comuna
primer_boton = resultados.find_element(By.XPATH, "./li[1]/button")
primer_boton.click()


# desplegar la lista de tipo de contrato y clickear el tipo deseado
lista_tipo_contrato = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, ":R2l5r:-trigger"))
)
lista_tipo_contrato.click()
select_tipo_contrato = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, tipo_contrato_dict[TIPO_CONTRATO]))
)
select_tipo_contrato.click()

tipo_inmueble = TIPO_INMUEBLE.lower().replace('á', 'a').replace('é', 'e').\
    replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
lista_tipo_inmueble = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, ":R4l5r:-trigger"))
)
lista_tipo_inmueble.click()

select_tipo_inmueble = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, tipo_inmueble_dict[tipo_inmueble]))
)
select_tipo_inmueble.click()


# apretar buscar
buscador = driver.find_element('id', ':R355r:')
buscador.click()


### 1.2 Selección de restricciones

Seleccionar rango de precio, entre 500000 a 1300000 pesos.

In [6]:
ingreso_precio_min = driver.find_element('id', ':R5chjmee:')
ingreso_precio_min.send_keys(MONTO_MINIMO)

ingreso_precio_max = driver.find_element('id', ':R6chjmee:')
ingreso_precio_max.send_keys(MONTO_MAXIMO)
ingreso_precio_max.send_keys(Keys.RETURN)

### 1.3 Recorrer inmuebles

- Para recorrer las páginas se decide navegar entre las urls implícitas en el botón siguiente.
Esto porque al intentar clickear el botón a veces se bloquea y a veces no, lo cual es un comportamiento
intermitente no deseable

- Se agrega una columna temporal "Tipo Moneda" para saber que propiedades tienen el precio en UF

- Al igual que los links de las publicaciones, la información restante (título, dirección y precio) pueden
extraerse directamente sin navegar a cada página. Sin embargo, como se pide navegar a cada página, entonces
se extraen solo los links de forma directa y se navega a cada uno para obtener el resto de la información.

Es importante que esta forma de extraer la información es considerablemente más lenta.

In [7]:
from selenium.common.exceptions import InvalidArgumentException

titulos, precios, direcciones, links_props = [], [], [], []

for i in range(CANT_PAGINAS):
    # Lo mismo para los enlaces
    links_props_values = driver.find_elements(By.XPATH,  "//h2[@class='poly-box poly-component__title']/a")
    links_props += [l.get_attribute('href') for l in links_props_values]

    if i<CANT_PAGINAS-1: # hacer click en siguiente
        try:
            boton_siguiente = driver.find_element(By.XPATH, "//li[contains(@class, 'andes-pagination__button--next')]/a")
            url_siguiente = boton_siguiente.get_attribute("href")
            driver.get(url_siguiente)
        except InvalidArgumentException: # se piden mas paginas de las que hay
            pass

# se navega a cada url para obtener titulo, precio y direccion
for url in links_props:
    driver.get(url)
    titulo_texto = driver.find_element(By.XPATH, "//h1[@class='ui-pdp-title']").text
    precio_texto = driver.find_element(By.XPATH, "//span[@class='andes-money-amount__fraction']").text
    direccion_texto = driver.find_element(By.XPATH, 
                    "//div[@class='ui-pdp-media ui-vip-location__subtitle ui-pdp-color--BLACK']/div[@class='ui-pdp-media__body']/p[@class='ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--REGULAR ui-pdp-media__title']").text

    titulos.append(titulo_texto)
    precios.append(precio_texto)
    direcciones.append(direccion_texto)

data_scrapping = pd.DataFrame({
    "Título": titulos,
    "Precio": precios,
    "Dirección": direcciones,
    "Enlace": links_props
})

# diferenciar precios en UF
data_scrapping['Precio'] = data_scrapping['Precio'].str.replace(' ', '')
data_scrapping['Precio'] = data_scrapping['Precio'].str.replace('.', '')
data_scrapping['Precio'] = data_scrapping['Precio'].str.replace('$', '')
data_scrapping['Precio'] = data_scrapping['Precio'].str.replace('UF', '')
data_scrapping['Precio'] = data_scrapping['Precio'].astype(int)
data_scrapping['Tipo moneda'] = 'CLP'
out_borders = (data_scrapping['Precio']>MONTO_MAXIMO) | (data_scrapping['Precio']<MONTO_MINIMO)
data_scrapping.loc[out_borders, 'Tipo moneda'] = 'UF'
data_scrapping.head(5)

,Título,Precio,Dirección,Enlace,Tipo moneda
0,Spot Nueva Kennedy,19,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",https://portalinmobiliario.com/MLC-1555797337-...,UF
1,Vespucio Switch,974560,"Av. Américo Vespucio Sur 345, Metro Escuela Mi...",https://portalinmobiliario.com/MLC-1525161537-...,CLP
2,Augusto Leguía,815000,"Augusto Leguía Nte. 70, Barrio El Golf, Las Co...",https://portalinmobiliario.com/MLC-2648931672-...,CLP
3,Somma Asturias - Edificio Multifamily,790000,"Asturias 77, Metro Escuela Militar, Las Condes...",https://portalinmobiliario.com/MLC-1408634227-...,CLP
4,Dpto Amoblado Remodelado 2d+2b Los Militares C...,27,"Los Militares 4717, Metro Escuela Militar, Las...",https://portalinmobiliario.com/MLC-2814352114-...,UF


### Cantidad de propiedades encontradas

In [8]:
len(data_scrapping)

96

In [9]:
data_scrapping['Precio'].describe()

count    9.600000e+01
mean     5.264011e+05
std      4.761229e+05
min      1.600000e+01
25%      2.575000e+01
50%      6.700000e+05
75%      9.250000e+05
max      1.300000e+06
Name: Precio, dtype: float64

Como se puede ver, el máximo de precio se condice con la restricción y el mínimo no cumple debido a los precios en UF.

### 1.4 Valor UF

Buscamos el valor al día de la UF.

In [10]:
# Configuramos el driver para el navegador Edge
#driver = webdriver.Edge()
driver = webdriver.Safari()
driver.set_window_size(1200, 800)  # Ancho: 1200px, Alto: 800px

driver.get(URL_UF)

# Establecemos una espera implícita de 5 segundos para que la página cargue completamente
driver.implicitly_wait(10)
valor_uf = driver.find_element(By.XPATH, "//span[@class='vpr']").text
valor_uf = float(valor_uf.replace('$', '').replace(' ', '').replace('.', '').replace(',', '.'))
print(f'El valor de la UF actualizado es: {valor_uf}')

time.sleep(5)
driver.close() 


El valor de la UF actualizado es: 38421.65


### Convertimos los precios marcados como UF a CLP.

In [11]:
data_scrapping_procesada = data_scrapping.copy()
data_scrapping_procesada['Precio'] = data_scrapping_procesada.apply(lambda row: row['Precio']*valor_uf \
    if row['Tipo moneda'] == 'UF' else row['Precio'], axis = 1)
data_scrapping_procesada['Precio'] = data_scrapping_procesada['Precio'].astype(int)
data_scrapping_procesada.drop(['Tipo moneda'], axis = 1, inplace = True)
data_scrapping_procesada = data_scrapping_procesada[(data_scrapping_procesada['Precio']>=MONTO_MINIMO) & (data_scrapping_procesada['Precio']<=MONTO_MAXIMO)]
data_scrapping_procesada.head(5)

,Título,Precio,Dirección,Enlace
0,Spot Nueva Kennedy,730011,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",https://portalinmobiliario.com/MLC-1555797337-...
1,Vespucio Switch,974560,"Av. Américo Vespucio Sur 345, Metro Escuela Mi...",https://portalinmobiliario.com/MLC-1525161537-...
2,Augusto Leguía,815000,"Augusto Leguía Nte. 70, Barrio El Golf, Las Co...",https://portalinmobiliario.com/MLC-2648931672-...
3,Somma Asturias - Edificio Multifamily,790000,"Asturias 77, Metro Escuela Militar, Las Condes...",https://portalinmobiliario.com/MLC-1408634227-...
4,Dpto Amoblado Remodelado 2d+2b Los Militares C...,1037384,"Los Militares 4717, Metro Escuela Militar, Las...",https://portalinmobiliario.com/MLC-2814352114-...


In [12]:
data_scrapping_procesada['Precio'].describe()

count    9.600000e+01
mean     9.034035e+05
std      2.042327e+05
min      5.550000e+05
25%      7.275000e+05
50%      9.000000e+05
75%      1.075806e+06
max      1.300000e+06
Name: Precio, dtype: float64

Se puede ver que ahora efectivamente, el precio está dentro de rango permitido de 550.000 hasta 1.300.000 pesos.

### 1.5 Procesamiento direcciones

Direcciones con problemas: intervalos numéricos sin dar la dirección exacta

Se consulta chat GPT por el pattern exacto a ocupar

In [13]:
# Expresión regular para buscar direcciones con un intervalo numérico
pattern = r'\b(\d+(?:\.\d+)?(?:e[+-]?\d+)?)\s*-\s*(\d+(?:\.\d+)?(?:e[+-]?\d+)?)\b'
data_scrapping_procesada[data_scrapping_procesada['Dirección'].str.contains(pattern, flags=re.IGNORECASE, regex=True)].head(10)

/var/folders/mf/j5c_wycn5cdgbkl59yvppg500000gn/T/ipykernel_1461/558544235.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data_scrapping_procesada[data_scrapping_procesada['Dirección'].str.contains(pattern, flags=re.IGNORECASE, regex=True)].head(10)


,Título,Precio,Dirección,Enlace
15,"Guanabara 2d2b + Est Y Bdga, Las Condes",1180000,"Guanabara 900 - 1200, Las Condes, Rotonda Aten...",https://portalinmobiliario.com/MLC-2814805660-...
21,Arrienda Studio Nuevo Amoblado Moderno Edifici...,690000,"Las Tranqueras 1 - 300, Las Condes, Alto Las C...",https://portalinmobiliario.com/MLC-2812221716-...
29,Metro Escuela Militar/albacete/2 Dormitorios/2...,1075806,"Albacete 4200 - 4500, Metro Escuela Militar, L...",https://portalinmobiliario.com/MLC-1564396001-...
34,"Oportunidad, Departamento De 1 Dormitorio Con ...",700000,"Hnos Cabot 7800 - 8100, Las Condes, Alto Las C...",https://portalinmobiliario.com/MLC-2814626488-...
41,Excelente Depto. Un Dorm. Amoblado A Pasos Me...,730000,"Evaristo Lillo 29, Depto 1 - 300, Las Condes, ...",https://portalinmobiliario.com/MLC-2805523956-...
42,Excelente Depto. 1 Dorm. Amoblado Metro Alcantara,940000,"Alsacia 1 - 300, Las Condes, Barrio El Golf, L...",https://portalinmobiliario.com/MLC-1560527075-...
49,Dpto Amoblado De 1d1b Con Estacionamiento En L...,700000,"Las Verbenas 7800 - 8100, Las Condes, Alto Las...",https://portalinmobiliario.com/MLC-2800458820-...
50,"Depto Nuevo 2d Mariposa, Terraza Y Amenities E...",1152649,"Av. Las Condes 10500 - 10800, Las Condes, Esto...",https://portalinmobiliario.com/MLC-2814786802-...
52,"Nuevo 2d Mariposa, Terraza, Piscina Y Quincho,...",1152649,"Av. Las Condes 10500 - 10800, Las Condes, Esto...",https://portalinmobiliario.com/MLC-1562477369-...
57,Departamento Familiar A Pasos De Rotonda Atenas,1200000,"Robinson Crusoe 900 - 1200, Las Condes, Rotond...",https://portalinmobiliario.com/MLC-2818345382-...


Se corrige este tipo de direcciones tomando el promedio del intervalo que se da en la dirección

In [14]:
def corregir_direccion(direccion):
    match = re.search(pattern, direccion)
    if match:
        inicio, fin = map(float, match.groups())
        promedio = (inicio + fin) / 2
        return re.sub(pattern, f"{promedio:.0f}", direccion)
    else:
        return direccion


# Crear nueva dirección procesada
data_scrapping_procesada['Dirección'] = data_scrapping_procesada['Dirección'].apply(corregir_direccion)
data_scrapping_procesada[data_scrapping_procesada['Dirección'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]



/var/folders/mf/j5c_wycn5cdgbkl59yvppg500000gn/T/ipykernel_1461/2078627988.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data_scrapping_procesada[data_scrapping_procesada['Dirección'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]


,Título,Precio,Dirección,Enlace


In [15]:
data_scrapping_procesada.head(5)

,Título,Precio,Dirección,Enlace
0,Spot Nueva Kennedy,730011,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",https://portalinmobiliario.com/MLC-1555797337-...
1,Vespucio Switch,974560,"Av. Américo Vespucio Sur 345, Metro Escuela Mi...",https://portalinmobiliario.com/MLC-1525161537-...
2,Augusto Leguía,815000,"Augusto Leguía Nte. 70, Barrio El Golf, Las Co...",https://portalinmobiliario.com/MLC-2648931672-...
3,Somma Asturias - Edificio Multifamily,790000,"Asturias 77, Metro Escuela Militar, Las Condes...",https://portalinmobiliario.com/MLC-1408634227-...
4,Dpto Amoblado Remodelado 2d+2b Los Militares C...,1037384,"Los Militares 4717, Metro Escuela Militar, Las...",https://portalinmobiliario.com/MLC-2814352114-...


# 2. Uso de Google APIs

Para la siguiente parte utilizaremos las APIs: Geocoding y places

### Geocoding

In [16]:
# Función para obtener datos de geocoding
def obtener_datos_geocoding(direccion):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={direccion}&key={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data["results"]:
            geometry = data["results"][0]["geometry"]
            location = geometry["location"]
            place_id = data["results"][0]["place_id"]
            return {'geometry': {'location': location}, 'place_id': place_id}
    return {'geometry': {'location': {'lat': None, 'lng': None}}, 'place_id': None, 'rating': None, 'user_ratings_total': None}

# Añadir datos al DataFrame
data_scrapping_procesada['Geocoding_Data'] = data_scrapping_procesada['Dirección'].apply(obtener_datos_geocoding)

# Separar columnas para latitud, longitud y place_id
data_scrapping_procesada['Latitud'] = data_scrapping_procesada['Geocoding_Data'].apply(lambda x: x['geometry']['location']['lat'])
data_scrapping_procesada['Longitud'] = data_scrapping_procesada['Geocoding_Data'].apply(lambda x: x['geometry']['location']['lng'])
data_scrapping_procesada['Place_ID'] = data_scrapping_procesada['Geocoding_Data'].apply(lambda x: x['place_id'])
data_scrapping_procesada.drop(['Geocoding_Data'], axis = 1, inplace=True)

In [17]:
data_scrapping_procesada.head(5)

,Título,Precio,Dirección,Enlace,Latitud,Longitud,Place_ID
0,Spot Nueva Kennedy,730011,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",https://portalinmobiliario.com/MLC-1555797337-...,-33.399199,-70.571350,ChIJ3bQIbtPPYpYRHdV3dzX_lYM
1,Vespucio Switch,974560,"Av. Américo Vespucio Sur 345, Metro Escuela Mi...",https://portalinmobiliario.com/MLC-1525161537-...,-33.417402,-70.583612,ChIJtcMsohnPYpYR_IXSuaKSuuE
2,Augusto Leguía,815000,"Augusto Leguía Nte. 70, Barrio El Golf, Las Co...",https://portalinmobiliario.com/MLC-2648931672-...,-33.416026,-70.597734,ChIJV8yNVBXPYpYRNw_Po6puQ7c
3,Somma Asturias - Edificio Multifamily,790000,"Asturias 77, Metro Escuela Militar, Las Condes...",https://portalinmobiliario.com/MLC-1408634227-...,-33.415309,-70.585786,ChIJHabRUejPYpYRLoiRk3yQ9rg
4,Dpto Amoblado Remodelado 2d+2b Los Militares C...,1037384,"Los Militares 4717, Metro Escuela Militar, Las...",https://portalinmobiliario.com/MLC-2814352114-...,-33.411847,-70.580217,ChIJJ41HGSLPYpYRh0vwgaF4Pig


Casos no detectados con Geocoding

In [18]:
data_scrapping_procesada[data_scrapping_procesada["Place_ID"].isnull()]

,Título,Precio,Dirección,Enlace,Latitud,Longitud,Place_ID


### Places

In [19]:
# Función para buscar lugares cerca de una ubicación según el rubro
def buscar_lugares(lat, lng, rubro, radio=RADIO_BUSQUEDA):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lng}",
        "radius": radio,
        "type": rubro,
        "key": API_KEY  # Asegúrate de tener tu API_KEY
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if "results" in data:
            return [(result["name"], result["place_id"], 
                     result["rating"] if "rating" in result else None, 
                     result["user_ratings_total"] if "user_ratings_total" in result else None
                     ) for result in data["results"]]
    return []

# Crear un nuevo DataFrame para almacenar los resultados
resultados = []

# Lista de rubros a buscar
busqueda_rubros = BUSQUEDA_RUBROS  # Tipos de lugares
radio_busqueda = RADIO_BUSQUEDA  # Radio en metros

# Iterar sobre cada fila del DataFrame original
for index, row in data_scrapping_procesada.iterrows():
    direccion_original = row['Dirección']
    lat, lng, place_id_original = row['Latitud'], row['Longitud'], row['Place_ID']
    
    # Buscar lugares para cada rubro
    for rubro in busqueda_rubros:
        lugares = buscar_lugares(lat, lng, rubro, radio_busqueda)
        for lugar in lugares:
            resultados.append({
                "Direccion_Original": direccion_original,
                "Place_ID_Original": place_id_original,
                "Rubro": rubro,
                "Nombre_Lugar": lugar[0],
                "Place_ID_Lugar": lugar[1],
                "Rating": lugar[2],
                "User_Ratings_Total": lugar[3]
            })

# Convertir los resultados a un DataFrame
df_lugares = pd.DataFrame(resultados)

In [20]:
df_lugares.head(5)

,Direccion_Original,Place_ID_Original,Rubro,Nombre_Lugar,Place_ID_Lugar,Rating,User_Ratings_Total
0,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",ChIJ3bQIbtPPYpYRHdV3dzX_lYM,restaurant,Latin Grill,ChIJ7_fk2NfOYpYRvKmszamPz9g,4.6,374.0
1,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",ChIJ3bQIbtPPYpYRHdV3dzX_lYM,restaurant,Shawerma,ChIJLXfz627PYpYRXX4NU9Iugks,4.1,9.0
2,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",ChIJ3bQIbtPPYpYRHdV3dzX_lYM,restaurant,Restaurante Messers,ChIJ_cRizEnPYpYRxgMIf7AWZNA,4.1,81.0
3,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",ChIJ3bQIbtPPYpYRHdV3dzX_lYM,restaurant,K-fe,ChIJ575n4tfOYpYRRnv16mdOiJ0,4.0,1.0
4,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",ChIJ3bQIbtPPYpYRHdV3dzX_lYM,grocery_or_supermarket,Spid: Miles de productos,ChIJ615L_AnPYpYRBmI-sv0NwLE,2.7,20.0


# 3  Gestión de Datos

### 3.1 Creación de tablas

El Place_ID puede repetirse, ya que hay arriendos de un mismo edificio y por tanto tienen la misma dirección. Se usa (Titulo, Precio, Direccion) como PRIMARY KEY

In [21]:
# Crear conexión a la base de datos SQLite
conn = sqlite3.connect('trabajo_final_ACD.db')
cursor = conn.cursor()

# Definir el esquema de la base de datos
cursor.executescript('''
BEGIN TRANSACTION;
DROP TABLE IF EXISTS inmuebles;
CREATE TABLE IF NOT EXISTS inmuebles (
    Titulo TEXT,
    Precio REAL,
    Direccion TEXT,
    Enlace TEXT,
    Latitud REAL,
    Longitud REAL,
    Place_ID TEXT,
    PRIMARY KEY (Titulo, Precio, Direccion)
);
COMMIT;               
''')

cursor.executescript('''
BEGIN TRANSACTION;
DROP TABLE IF EXISTS lugares_cercanos;
CREATE TABLE IF NOT EXISTS lugares_cercanos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Direccion_Original TEXT,
    Place_ID_Original TEXT,
    Rubro TEXT,
    Nombre_Lugar TEXT,
    Place_ID_Lugar TEXT,
    Rating REAL,
    User_Ratings_Total INTEGER,
    FOREIGN KEY (Direccion_Original) REFERENCES inmuebles (Direccion)
    FOREIGN KEY (Place_ID_Original) REFERENCES inmuebles (Place_ID)
                     
);
COMMIT;
''')

# Confirmar creación de tablas
conn.commit()

Cargamos datos a las tablas desde los dataframes

In [22]:
# Insertar datos del DataFrame "data_scrapping_procesada" en la tabla "inmuebles"
for index, row in data_scrapping_procesada.iterrows():
    cursor.execute('''
    INSERT INTO inmuebles (Titulo, Precio, Direccion, Enlace, Latitud, Longitud, Place_ID)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (row['Título'], row['Precio'], row['Dirección'], row['Enlace'], row['Latitud'], row['Longitud'], row['Place_ID']))

# Insertar datos del DataFrame "df_lugares" en la tabla "lugares_cercanos"
for index, row in df_lugares.iterrows():
    cursor.execute('''
    INSERT INTO lugares_cercanos (Direccion_Original, Place_ID_Original, Rubro, Nombre_Lugar, Place_ID_Lugar, Rating, User_Ratings_Total)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (row['Direccion_Original'], row['Place_ID_Original'], row['Rubro'], row['Nombre_Lugar'], row['Place_ID_Lugar'], row['Rating'], row['User_Ratings_Total']))

# Confirmar inserción de datos
conn.commit()

conn.close()

### 3.2 Consultas

¿Cuál es el valor promedio de los 20 arriendos de dpto más baratos en Las Condes?

In [23]:
# Conectar a la base de datos
conn = sqlite3.connect('trabajo_final_ACD.db')
cursor = conn.cursor()

# Consulta SQL
query = '''
SELECT AVG(Precio) AS Promedio_Precio
FROM (
    SELECT Precio
    FROM inmuebles
    ORDER BY Precio ASC
    LIMIT 20
) AS top_20_baratos;
'''

# Ejecutar la consulta
cursor.execute(query)
resultado = cursor.fetchone()

# Mostrar el resultado
print(f"El valor promedio de los 20 arriendos más baratos es: ${resultado[0]:.2f} CLP")

# Cerrar la conexión
conn.close()

El valor promedio de los 20 arriendos más baratos es: $651596.55 CLP


¿Cuál es la mediana de comentarios (user_ra ngs_total), de aquellos lugares cercanos,
que enen una valoración igual o superior a 4 estrellas y que corresponden a los 15
dptos más baratos de Las Condes?

In [24]:
# Conectar a la base de datos
conn = sqlite3.connect('trabajo_final_ACD.db')
cursor = conn.cursor()

# Consulta SQL
query = '''

-- los departamentos más baratos
WITH top_15_baratos AS (
SELECT *
FROM inmuebles
ORDER BY Precio ASC
LIMIT 15
),

-- user rantings de los 15 mas baratos
ratings AS (
    SELECT l.User_Ratings_Total
    FROM top_15_baratos t
    LEFT JOIN lugares_cercanos l
    ON t.Place_ID = l.Place_ID_Original
    AND t.Direccion = l.Direccion_Original
    AND l.Rating >= 4
    ORDER BY l.User_Ratings_Total
),

-- ordenar user ratings
ranked_ratings AS (
    SELECT User_Ratings_Total,
           ROW_NUMBER() OVER () AS row_num,
           COUNT(*) OVER () AS total_count
    FROM ratings
)

-- calcular mediana
SELECT 
    CASE
        WHEN total_count % 2 = 1 THEN 
            -- Si el número de elementos es impar, selecciona el valor central
            (SELECT User_Ratings_Total FROM ranked_ratings WHERE row_num = (total_count + 1) / 2)
        ELSE
            -- Si el número de elementos es par, promedia los dos valores centrales
            (
                SELECT AVG(User_Ratings_Total) 
                FROM ranked_ratings 
                WHERE row_num IN (total_count / 2, total_count / 2 + 1)
            )
    END AS mediana
FROM ranked_ratings
LIMIT 1;
'''

# Ejecutar la consulta
cursor.execute(query)
resultado = cursor.fetchone()
print(f"Mediana de usuarios que puntúan con al menos 4 estrellas los lugares cercanos de los 15 departamentos más baratos: {resultado[0]:.2f}")
conn.close()

Mediana de usuarios que puntúan con al menos 4 estrellas los lugares cercanos de los 15 departamentos más baratos: 106.00


# Conclusión

Este trabajo nos permite consolidar los conocimientos prácticos en web scraping, manejo de APIs y gestión de bases de datos, aplicados a un problema real de consulta de inmuebles. La integración de herramientas como Selenium y las APIs de Google fomenta el desarrollo de habilidades técnicas avanzadas, mientras que la organización de los datos en una base relacional asegura un enfoque estructurado para el análisis.


Destacamos una planificación eficiente, el uso responsable de recursos y la originalidad en la implementación, preparandonos para enfrentar desafíos similares en el ámbito profesional.